Some links that helped me:<br>
https://medium.com/swlh/using-and-calling-an-api-with-python-494a18cb1f44 <br>
https://fdc.nal.usda.gov/api-guide.html<br>
https://fdc.nal.usda.gov/api-spec/fdc_api.html#/FDC/getFood<br>
https://api.data.gov/docs/api-key/

In [ ]:
import requests
import bs4
import pandas as pd
import time
from sys import argv

In [ ]:
script, csv_file = argv
USDA_API_KEY = '' # Your api goes here
base_url = 'https://api.nal.usda.gov/fdc/v1/food/'
headers = {"X-Api-Key": USDA_API_KEY, "Content-type": "application/json"}
#            "X-RateLimit-Limit" : '' , 'X-RateLimit-Remaining' : ''}

In [ ]:
# Fuction to get the json file from USDA
def get_json(fdc_id):
    # Full url to get data
    full_url = base_url + fdc_id
    # using headers from above
    # most important is the API key
    res = requests.get(full_url,headers=headers)
    json = res.json()
    return json

In [ ]:
# Fuction to gather all the nutrition data from food item on USDA
def gather_data_to_df(json):
    # Gathering a list of dictionaries
    all_nutrient_data = []
    
    # for loop to gather all information
    for num in range(len(json['foodNutrients'])):
        # had to create an empty amount variable due to error
        # some values didn't have an amount key from the json file
        amount = ''
        nutrient = json['foodNutrients'][num]['nutrient']['name']
        try:
            amount = json['foodNutrients'][num]['amount']
        except:
            pass
        unit = json['foodNutrients'][num]['nutrient']['unitName']
        # creating a dictionary with all the information
        nutrient_data ={
            'food_id': json['fdcId'],
            'food-item': json['description'],
            'ndb_number': json['ndbNumber'],
            'nutrient': nutrient,
            'amount': amount,
            'unit': unit
        }
        all_nutrient_data.append(nutrient_data)
        
        # turning all the information into a dataframe
        food_item_df = pd.DataFrame(all_nutrient_data)
        
    return food_item_df 

In [ ]:
# Not all the information is needed
# This fuction is to only keep the nutrition facts I need
def specific_nutrients(full_item_df):
    specific_nutrients_df = full_item_df[(full_item_df['nutrient'] == 'Protein') | 
              ((full_item_df['nutrient'] == 'Energy') & (full_item_df['unit'] == 'kcal')) | 
              (full_item_df['nutrient'] == 'Calcium, Ca') |
              (full_item_df['nutrient'] == 'Carbohydrate, by difference') | 
              (full_item_df['nutrient'] == 'Cholesterol') |
              (full_item_df['nutrient'] == 'Fatty acids, total saturated') |
              (full_item_df['nutrient'] == 'Fatty acids, total trans') |
              (full_item_df['nutrient'] == 'Iron, Fe') |
              (full_item_df['nutrient'] == 'Potassium, K') |
              (full_item_df['nutrient'] == 'Sodium, Na') |
              (full_item_df['nutrient'] == 'Fatty acids, total trans') |
              (full_item_df['nutrient'] == 'Sugars, total including NLEA') | 
              (full_item_df['nutrient'] == 'Sugars, added') |
              (full_item_df['nutrient'] == 'Total lipid (fat)') | 
              (full_item_df['nutrient'] == 'Vitamin A, IU') | 
              (full_item_df['nutrient'] == 'Vitamin A, RAE') |
              (full_item_df['nutrient'] == 'Vitamin C, total ascorbic acid') |
              (full_item_df['nutrient'] == 'Vitamin D (D2 + D3), International Units') |
              (full_item_df['nutrient'] == 'Vitamin D (D2 + D3)') |
              (full_item_df['nutrient'] == 'Fiber, total dietary')
             ]
    return specific_nutrients_df

In [ ]:
# Re-indexing by nutrient
def new_index_order(clean_df):
    new_index_df = clean_df.set_index(['nutrient'])
    final_df = new_index_df.reindex(['Calcium, Ca', 'Energy', 'Carbohydrate, by difference', 'Cholesterol', 
                             'Fatty acids, total saturated', 'Fatty acids, total trans', 'Fiber, total dietary',
                             'Iron, Fe', 'Potassium, K', 'Protein', 'Sodium, Na', 'Sugars, added', 
                             'Sugars, total including NLEA', 'Total lipid (fat)', 'Vitamin A, IU', 
                             'Vitamin A, RAE', 'Vitamin C, total ascorbic acid', 
                             'Vitamin D (D2 + D3), International Units', 'Vitamin D (D2 + D3)'])
    return final_df

In [ ]:
# Opening a csv file to read in all the FDC-ID codes

# Empty list to gather all FDC_ID codes
fdc_id_list = []
with open(csv_file, 'r') as file:
    for line in file:
        fdc_id_list.append(line.strip())

In [ ]:
# Final bit of code putting all the functions together
# Added printing statements to know where in the process the script is

all_dataframes = []
for fdc_id in fdc_id_list:
    print(f"Getting json for FDC ID: {fdc_id}")
    json = get_json(fdc_id)
    food_item = json['description']
    print(f"Now getting all nutrients for '{food_item}'")
    full_nutrients = gather_data_to_df(json)
    print("Removing unnecessary nutrients")
    specific_nutrients_df = specific_nutrients(full_nutrients)
    print("Re-ordering nutrients")
    final_df = new_index_order(specific_nutrients_df)
    all_dataframes.append(final_df)
    print(f"'{food_item}' done!")
    print()
    print('*')
    time.sleep(1)
    print('**')
    time.sleep(1)
    print('***')
    time.sleep(1)
    print('****')
    time.sleep(1)
    print('*****')
    
print("*"*100)    
print("Combining all dataframes")
all_df = pd.concat(all_dataframes)
print("Exporting a CSV file")
all_df.to_csv(all_df['food-item'][0] + '.csv')
print("**********")
print("ALL DONE!!")
print("**********")

# Nutrient values needed for ChefTec (In order)
- Calcium, Ca
- Calories
- Carbohydrate by difference
- Cholesterol
- Fatty acids, saturated
- Fatty acids, total trans
- Fiber, total dietary
- Iron, Fe
- Potassium, K (new label)
- Protein
- Sodium, Na
- Sugars, added
- Sugars, total
- Total lipid (fat)
- Vitamin A, IU (old label)
- Vitamin A, RAE (new label)
- Vitamin C (old label)
- Vitamin D (D2 + D3) (new label)
- Vitmain D (old label)